<a href="https://colab.research.google.com/github/chorltonm/fa-cup-upsets/blob/main/notebooks/models/ensemble_random_forest_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import general python libaries
import os
import json
import pandas as pd
import numpy as np
import importlib

# Google Cloud libraries
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import drive
from google.colab import userdata
import pandas_gbq

# Scikit Learn libraries
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Other
from matplotlib import pyplot
import seaborn as sns



In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Change default ouput directory
os.chdir('/content/drive/MyDrive/birkbeck_msc-project/python_files')



Mounted at /content/drive


In [ ]:
# Import user defined python functions
import model_evaluation_functions
importlib.reload(model_evaluation_functions)
from model_evaluation_functions import create_model_results_df

In [ ]:
# Authentication credentials and keys

# Google Service Account

# Load the JSON key from local Google Collab file
key = json.load(open('/content/drive/MyDrive/service_account.json', 'r'))

# Authenticate using the loaded key
credentials = service_account.Credentials.from_service_account_info(key)

# Set up the BigQuery client with the credentials to project
client = bigquery.Client(credentials=credentials, project='birkbeck-msc-project-422917')

In [ ]:
# Feature Data

fa_cup_raw_features = """
    select * from preparation_layer.view_fa_cup_round_3_features
"""

fa_cup_raw_features_df = client.query(fa_cup_raw_features).to_dataframe()
display(fa_cup_raw_features_df)


all_results_df = pd.DataFrame(columns=['metric_id', 'metric'])
display(all_results_df)


,arena_stadium_capacity,home_team_league_level,home_win,home_team_round_3_position,home_team_massey,home_team_colley,home_team_keener,home_team_trueskill,home_team_borda_count,home_team_local_kemeny_optimisation,...,match_weather_data_hours_dew,match_weather_data_hours_precip,match_weather_data_hours_snow,match_weather_data_hours_snowdepth,match_weather_data_hours_windspeed,match_weather_data_hours_pressure,match_weather_data_hours_visibility,match_weather_data_hours_cloudcover,match_weather_data_hours_condition,target_variable
0,41837,1,1,1,1,1,12,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0
1,41837,1,1,1,1,1,12,1,1,1,...,34.70,0.000,0.0,0.00,4.80,1036.70,2.05,95.90,Overcast,0
2,41837,1,1,1,1,1,12,1,1,1,...,47.00,0.000,0.0,0.00,2.10,1031.70,4.20,87.00,Partially cloudy,0
3,74879,1,1,1,1,1,12,1,1,1,...,71.20,0.000,NaN,0.00,7.65,1015.50,5.15,67.10,Partially cloudy,0
4,74879,1,0,2,3,7,4,6,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,<NA>,6,0,64,58,64,57,63,64,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0
301,<NA>,6,0,64,58,64,57,63,64,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0
302,9527,4,1,64,58,64,57,63,64,64,...,37.15,0.000,0.0,0.00,14.45,1012.75,9.00,94.25,Overcast Partially cloudy,1
303,6064,6,0,64,58,64,57,63,64,64,...,41.45,0.000,0.0,0.00,6.35,1036.35,15.15,98.65,Overcast,0


,metric_id,metric


In [ ]:
def add_home_advantage(X):

    result = X.groupby(['home_team_league_level', 'away_team_league_level'])['home_win'].mean().reset_index()
    result.columns = ['home_team_league_level', 'away_team_league_level', 'home_win_factor']
    result['home_win_factor'] = result['home_win_factor'].round(3)

    X = X.merge(result, on=['home_team_league_level', 'away_team_league_level'], how='left')
    return X, 'home_win_factor'

In [ ]:
# Random Forest Classifier with home advantage
model_name = "Random Forest Classifier with home advantage"
random_state = 47

# Empty df for consolidating confusion report results
all_results_df_rf = pd.DataFrame(columns=['metric_id', 'metric'])

# Define the ranking systems
ranking_systems = ['no_ranking', 'round_3_position', 'massey', 'colley', 'keener', 'trueskill', 'borda_count', 'local_kemeny_optimisation']

# Get all columns except the target and ranking columns
target_variable = 'target_variable'
base_features = [col for col in fa_cup_raw_features_df.columns if col != target_variable and not any(f"{team}_{ranking}" in col for team in ['home_team', 'away_team'] for ranking in ranking_systems)]

# Identify numeric and categorical columns
numeric_features = fa_cup_raw_features_df[base_features].select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = fa_cup_raw_features_df[base_features].select_dtypes(include=['object']).columns.tolist()

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

ranking_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

# Define cross-validation strategy
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

# Loop through each ranking system
for ranking in ranking_systems:
    model_name_ranking = f"{model_name} {ranking}"
    print(model_name_ranking)

    if ranking == 'no_ranking':
        # Use all base features without ranking
        X = fa_cup_raw_features_df[base_features]
    else:
        # Prepare the feature set for the current ranking system to include ranking features
        current_features = base_features + [f'home_team_{ranking}', f'away_team_{ranking}']
        X = fa_cup_raw_features_df[current_features]

    y = fa_cup_raw_features_df['target_variable']

    # Initialize lists to store results for each fold of the model
    fold_accuracies = []
    fold_recalls = []
    fold_results = []
    confusion_matrices = []
    classification_reports = []
    all_y_true = []
    all_y_pred = []
    all_y_pred_proba = []  # Store probabilities for ROC

    # Perform cross-validation
    for fold, (train_index, val_index) in enumerate(skf.split(X, y), 1):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Add home advantage factor
        X_train, home_advantage_column = add_home_advantage(X_train)
        X_val, _ = add_home_advantage(X_val)

        # Update numeric_features to include the home advantage column
        numeric_features_with_ha = [home_advantage_column] + numeric_features

        # Update preprocessor with current features including home advantage
        if ranking == 'no_ranking':
            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features)
                ], verbose_feature_names_out=False)
        else:
            preprocessor = ColumnTransformer(
                transformers=[
                    ('num', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features),
                    ('rank', numeric_transformer, [f'home_team_{ranking}', f'away_team_{ranking}'])
                ], verbose_feature_names_out=False)

        # Create pipeline
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(random_state=random_state))
        ])

        # Define parameter grid for GridSearchCV
        param_grid = {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__min_samples_leaf': [1, 2, 4],
            'classifier__max_features': [ None, 'sqrt']
        }

        # Create GridSearchCV object
        grid_search = GridSearchCV(pipeline, param_grid, cv=2, scoring='recall', n_jobs=-1)

        # Fit GridSearchCV
        grid_search.fit(X_train, y_train)

        # Get best model
        best_model = grid_search.best_estimator_

        # Print the best recall from GridSearchCV
        print(f"Best parameters: {grid_search.best_params_}")
        print(f"Best Estimator's Recall (from GridSearchCV): {grid_search.best_score_:.3f}")

        # Convert cv_results_ to DataFrame
        #cv_results_df = pd.DataFrame(grid_search.cv_results_)
        #display(cv_results_df)

        # Make predictions
        y_pred = best_model.predict(X_val)
        y_pred_proba = best_model.predict_proba(X_val)[:, 1]

        # Calculate accuracy
        accuracy = accuracy_score(y_val, y_pred)
        fold_accuracies.append(accuracy)

        # Calculate recall
        recall = recall_score(y_val, y_pred)
        fold_recalls.append(recall)

         # Store fold results
        fold_results.append({
            'Fold': fold,
            'Best Parameters (GridSearchCV)': grid_search.best_params_,
            'Best Recall (GridSearchCV)': grid_search.best_score_,
            'Validation Accuracy': accuracy,
            'Validation Recall': recall
        })

        # Confusion matrix
        cm = confusion_matrix(y_val, y_pred)
        confusion_matrices.append(cm)

        # Generate classification report
        cr = classification_report(y_val, y_pred, output_dict=True)
        classification_reports.append(cr)

        # Store true and predicted values for later analysis
        all_y_true.extend(y_val)
        all_y_pred.extend(y_pred)
        all_y_pred_proba.extend(y_pred_proba)

        print(f"Fold {fold} Accuracy: {accuracy:.3f}")
        print(f"Fold {fold} Recall: {recall:.3f}")

    # Calculate average accuracy
    avg_accuracy = sum(fold_accuracies) / len(fold_accuracies)
    print(f"Average Accuracy: {avg_accuracy:.3f}")

    # Calculate average recall
    avg_recall = sum(fold_recalls) / len(fold_recalls)
    print(f"Average Recall: {avg_recall:.3f}")

    # Create and display results using existing function
    results_df, cm_fig, roc_fig = create_model_results_df(all_y_true, all_y_pred, fold_accuracies, all_y_pred_proba, model_name_ranking)
    results_df = results_df.reset_index()
    results_df['metric_id'] = results_df.index + 1
    results_df = results_df[['metric_id', 'metric', model_name_ranking]]

    all_results_df_rf = all_results_df_rf.merge(results_df, on=['metric_id', 'metric'], how='outer')

display(all_results_df_rf)

# Change default output directory
os.chdir('/content/drive/MyDrive/birkbeck_msc-project/output_files')
# Save results to excel
all_results_df_rf.to_excel("all_rf_results.xlsx")



,metric_id,metric,Random Forest Classifier with home advantage no_ranking,Random Forest Classifier with home advantage round_3_position,Random Forest Classifier with home advantage massey,Random Forest Classifier with home advantage colley,Random Forest Classifier with home advantage keener,Random Forest Classifier with home advantage trueskill,Random Forest Classifier with home advantage borda_count,Random Forest Classifier with home advantage local_kemeny_optimisation
0,1,Cross-validation Accuracy,0.882,0.885,0.882,0.869,0.908,0.872,0.892,0.918
1,2,Cross-validation Standard Deviation,0.059,0.059,0.042,0.027,0.037,0.035,0.047,0.041
2,3,Overall Accuracy,0.882,0.885,0.882,0.869,0.908,0.872,0.892,0.918
3,4,Confusion Matrix - True Negative (Class 0),220.000,222.000,221.000,218.000,220.000,219.000,220.000,220.000
4,5,Confusion Matrix - False Positive (Class 1),3.000,1.000,2.000,5.000,3.000,4.000,3.000,3.000
5,6,Confusion Matrix - False Negative (Class 0),33.000,34.000,34.000,35.000,25.000,35.000,30.000,22.000
6,7,Confusion Matrix - True Positive (Class 1),49.000,48.000,48.000,47.000,57.000,47.000,52.000,60.000
7,8,Precision (Class 0),0.870,0.867,0.867,0.862,0.898,0.862,0.880,0.909
8,9,Recall (Class 0),0.987,0.996,0.991,0.978,0.987,0.982,0.987,0.987
9,10,F1-score (Class 0),0.924,0.927,0.925,0.916,0.940,0.918,0.930,0.946


In [ ]:
all_results_df_lr = all_results_df_slrh.merge(all_results_df_slr, on=['metric_id', 'metric'], how='outer')
all_results_df_slnn = all_results_df_nn.merge(all_results_df_lr, on=['metric_id', 'metric'], how='outer')
all_results_df = all_results_df_nnh.merge(all_results_df_slnn, on=['metric_id', 'metric'], how='outer')
display(all_results_df)

In [ ]:
# Load updated Confusion Matrix results to Google Big Query
load_dataset_name = 'analysis_layer'
load_table_name = 'lr_ann_model_confusion_matrix_results'
full_table_name = f"{load_dataset_name}.{load_table_name}"
pandas_gbq.to_gbq(all_results_df, full_table_name, project_id='birkbeck-msc-project-422917', if_exists='replace')